#  VGG networks
 While AlexNet (and its corresponding ConvNet design pattern) is considered the granddaddy of convolutional networks, the VGGNet (and its corresponding VGG
design pattern) is considered the father of formalizing a design pattern based on groups of convolutions.

The fundamental principles behind the VGG design pattern are as follows:
1. Grouping multiple convolutions into blocks, with the same number of filters
2. Progressively doubling the number of filters across blocks
3. Delaying pooling to the end of a block

When discussing a VGG design pattern in today’s context, initial confusion may arise over the terms group and block. In their research for VGGNet, the authors used the term convolutional group. Subsequently, researchers refined the grouping patterns into convolutional groups, consisting of convolutional blocks. In today’s nomenclature, the VGG group would be called a block.

 It is designed using a handful of principles that are easy to learn.
 1.The convolutional frontend consists of a sequence of pairs (and later triples) of convolutions of the same size, followed by a max pooling.
 2. The max pooling layer down-samples the generated feature maps by 75%, and
 3. the next pair (or triple) of convolutional layers then doubles the number of learned filters.


The principle behind the convolution design was that
1. the early layers learn coarse features, and
2. subsequent layers, by increasing the filters, learn finer and finer features, and
3. the max pooling is used between the layers to minimize growth in size (and subsequently parameters to learn) of the feature maps. Finally,
4. the DNN backend consists of two identically-sized dense hidden layers of 4096 nodes each, and a final dense output layer of 1000 nodes for classification.

<img src="img_10.png" >

The best-known versions are the VGG16 and VGG19. The VGG16 and VGG19 that were used in the competition, along with their trained weights from the competition,
were made publicly available. As they have been frequently used in transfer learning, others have kept the convolutional frontend of an ImageNet pretrained VGG16 or VGG19, and corresponding weights, and attached a new DNN backend for retraining for new classes of images.

<img src="img_11.png" width=1024>

So, let’s go ahead and code a VGG16 in two coding styles: the first in a sequential flow, and the second procedurally using reuse functions for duplicating the common blocks of layers, and parameters for their specific settings. We will also change specifying kernel_size and pool_size as keyword parameters and instead specify them as positional parameters

In [5]:
from keras import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input

In [3]:
VGG16 = Sequential([
    Conv2D(64, (3, 3), strides=(1, 1), padding="same",  activation="relu", input_shape=(224, 224, 3)),
    Conv2D(64,(3, 3), strides=(1, 1), padding="same",  activation="relu"),
    MaxPooling2D((2, 2), strides=(2, 2)),
    Conv2D(128, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    Conv2D(128, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    MaxPooling2D((2, 2), strides=(2, 2)),
    Conv2D(256, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    Conv2D(256, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    Conv2D(256, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    MaxPooling2D((2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    Conv2D(512, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    Conv2D(512, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    MaxPooling2D((2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    Conv2D(512, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    Conv2D(512, (3, 3), strides=(1, 1), padding="same",  activation="relu"),
    MaxPooling2D((2, 2), strides=(2, 2)),
    Flatten(),
    Dense(4096,  activation='relu'),
    Dense(4096,  activation='relu'),
    Dense(1000,  activation='softmax')
])
VGG16.compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")
VGG16.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

In [4]:
def conv_block(n_layers, n_filters, input_f):
    for i in range(n_layers):
        input_f = Conv2D(n_filters,  (3, 3), strides=(1, 1), padding="same",  activation="relu")(input_f)
    input_f = MaxPooling2D(2, strides=2)(input_f)
    return input_f

In [6]:
vgg_input = Input((224, 224, 3))
X = Conv2D(64,(3, 3), strides=(1, 1), padding="same",  activation="relu")(vgg_input)
X = conv_block(1, 64, X)
X = conv_block(2, 128, X)
X = conv_block(3, 256, X)
X = conv_block(3, 512, X)
X = conv_block(3, 512, X)
X = Flatten()(X)
X = Dense(4096)(X)
X = Dense(4096)(X)
outputs = Dense(1000)(X)
vgg16_model = Model(vgg_input, outputs)
vgg16_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_26 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_27 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_29 (Conv2D)          (None, 112, 112, 128)     147584    
                                                             